In [ ]:
# Extended source analysis
# for a spherical diffuser tip

In [ ]:
import numpy as np
from tables import get_c5, table4, table_a2, table_a5

In [ ]:
# Note, all time values are expressed in seconds [s]
# The analysis is for a repetitively pulsed laser
# emitting at a single wavelength

t_pulse = 80e-15      # pulse duration [s]
lambda_ = 784       # wavelength [nm]
f = 80.1e6      # laser rep rate [Hz]
# diffuser tip diameter [m]
r_sphere = 1e-3
# TODO make automatic determination based on class (and wavelength?)
t_base = 100     # time base for Class 1

In [ ]:
# note, the angular subtense of the apparent source
# depends on the aperture distance
# array of aperture distances from the sphere surface (reference point)
d_ap = np.arange(100e-3, 3500e-3, 1e-4)
# angular subtense of the apparent source (not limited just now)
alphas = 2 * r_sphere / (d_ap + r_sphere)

In [ ]:
## AEL

In [ ]:
### Single pulse analysis (determining AEL_single)
ael_single_j = np.array([table4(lambda_, t_pulse, alpha)[0] for alpha in alphas])

In [ ]:
### Average power analysis 
ael_t_w = np.array([table4(lambda_, t_base, alpha)[1] for alpha in alphas])
# see Note 8 of 4.3.f and Note 1 in Figure B.1
ael_spt_j = ael_t_w / f

In [ ]:
### Reduced pulse analysis (pulse train analysis) 
# get correction factor c5, the effective pulse duration,
# and the number of real pulses in the eff. pulse duration
# Note, alpha is kept at alpha_min for default evaluation
c5, t_eff, n = get_c5(lambda_, t_pulse, 1 / f, t_base, 1.5e-3)

# single pulse AEL for the effective pulse length [J]
ael_eff_j = np.array([table4(lambda_, t_eff, alpha)[0] for alpha in alphas])
# project back to an individual pulse and apply correction
ael_sptrain_j = ael_eff_j / n * c5

In [ ]:
aels = np.array([ael_single_j, ael_spt_j, ael_sptrain_j])
# minimum ael per aperture position
ael_d = np.min(aels, axis=0)

In [ ]:
# most restrictive positions [m]
# for Condition 1
d_min_c1 = 2000e-3  # minimum measuring distance for Condition 1
d_ael_c1 = d_ap[d_ap >= d_min_c1][np.argmin(ael_d[d_ap >= d_min_c1])]
# for Condition 3
d_min_c3 = 100e-3  # minimum measuring distance for Condition 3
d_ael_c3 = d_ap[d_ap >= d_min_c3][np.argmin(ael_d[d_ap >= d_min_c3])]
d_ael_c1, d_ael_c3

In [ ]:
# ael at most restrictive positions
ael_c1 = ael_d[np.where(d_ap == d_ael_c1)[0][0]]     # Condition 1
ael_c3 = ael_d[np.where(d_ap == d_ael_c3)[0][0]]     # Condition 3
ael_c1, ael_c3

In [ ]:
## Accessible emission 
p_avg = 0.124  # average laser power
w_c1 = 50e-3  # aperture diameter (Table 10)
w_c3 = 7e-3  # aperture diameter (Table 10)
# duty = t_pulse * f  # duty cycle of laser pulses
q_pulse = p_avg / f  # pulse energy

def ae(q, w, d):
    """
    Accessible emission from spherical diffuser
    :param q: pulse energy [J]
    :param w: aperture diameter [m]
    :param d: aperture distance from sphere surface [m]
    :return: accessible emission [J]
    """
    # distance from sphere centre
    d_c = d + 1e-3
    theta = np.atan(w / 2 / d_c)
    # pulse energy on aperture
    return q * (1 - np.cos(theta)) / 2

# pulse energy over aperture for Condition 1 and 3 
# at most restrictive position
q_single_c1 = ae(q_pulse, w_c1, d_ael_c1)
q_single_c3 = ae(q_pulse, w_c3, d_ael_c3)
# q_ap_single_c1, q_ap_single_c3

if q_single_c3 <= ael_c3:
    if q_single_c1 <= ael_c1:
        print('Product is Class 1')
    else:  # q_ap_single_c1 > ael, technically, we should check if it's also below class 3B
        print('Product is Class 1M')
else:
    print('Product is above Class 1, repeat test with higher class specs')

In [ ]:
## MPE for eye

In [ ]:
# locations for MPE and accessible emission evaluation 
# (from sphere surface)
r_mpe = np.arange(1e-7, 4e-3, 1e-7)
# angular subtense of the apparent source (not limited just now)
alphas_mpe = 2 * r_sphere / (r_mpe + r_sphere)

In [ ]:
alphas_mpe[:20]

In [ ]:
w_eye = 7e-3        # aperture diameter for the eye [m]
w_skin = 3.5e-3    # aperture diameter for the skin [m]

In [ ]:
### Single pulse analysis
mpe_s_eye_re = np.array([table_a2(lambda_, t_pulse, alpha)[0] for alpha in alphas_mpe])
# If the emission duration is shorter than 0.25s, 
# we need to check skin MPE as well to consider effects
# on the anterior parts of the eye (cornea, iris), 
# see note d) in Table A.2
if t_pulse < 0.25:
    mpe_s_skin_re = np.array([table_a5(lambda_, t_pulse)[0] for alpha in alphas_mpe])
    # We have two radiant exposures, 
    # averaged over apertures of different sizes.
    # To compare them, we project back 
    # how much energy this would be from the source.
    def ap_re_to_source(re, w, r):
        """
        :param re: radiant energy through aperture [J/m2]
        :param w: aperture diameter [m]
        :param r: aperture distance from sphere surface [m]
        :return: energy from the source [J]
        """
        a = (w/2)**2 * np.pi    # aperture area
        r_c = r + 1e-3  # distance from sphere centre
        theta = np.atan(w / 2 / r_c)
        return re * a / ((1 - np.cos(theta)) / 2)   # pulse energy from source
    
    q_eye = np.array(ap_re_to_source(mpe_s_eye_re, w_eye, r_mpe))
    q_skin = np.array(ap_re_to_source(mpe_s_skin_re, w_skin, r_mpe))
    
    # update eye MPEs with more restrictive skins MPEs
    mpe_s_eye_re[q_skin<q_eye] = mpe_s_skin_re[q_skin<q_eye]

In [ ]:
### Average power analysis 
mpe_avg_eye_i = np.array([table_a2(lambda_, t_base, alpha)[1] for alpha in alphas_mpe])
# see Note 8 of 4.3.f and Note 1 in Figure B.1
mpe_avg_eye_re = mpe_avg_eye_i / f
# Note, no need to check skin MPE, as the time base isn't below 0.25s

In [ ]:
### Reduced pulse analysis (pulse train analysis) 
# use correction factor c5, the effective pulse duration,
# and the number of real pulses in the eff. pulse duration

# single pulse MPE for the effective pulse length [J/m2]
mpe_eff_eye = np.array([table_a2(lambda_, t_eff, alpha)[0] for alpha in alphas_mpe])
# project back to an individual pulse and apply correction
mpe_sptrain_eye_re = mpe_eff_eye / n * c5
# MPE_sptrain is projected to a single pulse.
# If the pulse length is shorter than 0.25 s,
# we need to check single pulse skin MPE as well 
# to consider effects on the anterior parts 
# of the eye (cornea, iris), see note d) in Table A.2
if t_pulse < 0.25:
    q_eye = np.array(ap_re_to_source(mpe_sptrain_eye_re, w_eye, r_mpe))
    # update eye MPEs with more restrictive skins MPEs
    mpe_sptrain_eye_re[q_skin<q_eye] = mpe_s_skin_re[q_skin<q_eye]

In [ ]:
mpes_eye = np.array([mpe_s_eye_re, mpe_avg_eye_re, mpe_sptrain_eye_re])
# minimum mpe per aperture position
mpe_eye_d = np.min(mpes_eye, axis=0)

In [ ]:
## NOHD

In [ ]:
# at each aperture position, compare the emitted radiant exposure with the mpe
# array of accessible emission values
# (ae is vectorised, so can call it with array)
q_eye = ae(q_pulse, w_eye, r_mpe)
a_eye = (w_eye/2)**2 * np.pi
# array of radiant exposure values
re_eye = q_eye / a_eye

nohd = r_mpe[np.where(re_eye <= mpe_eye_d)[0][0]]
nohd

In [ ]:
## MPE for skin

In [ ]:
### Single pulse analysis
mpe_s_skin_re = np.array([table_a5(lambda_, t_pulse)[0] for alpha in alphas_mpe])

In [ ]:
### Average power analysis 
# t_skin = t_base
t_skin = 30000
mpe_avg_skin_i = np.array([table_a5(lambda_, t_skin)[0] for alpha in alphas_mpe])
# see Note 8 of 4.3.f and Note 1 in Figure B.1
mpe_avg_skin_re = mpe_avg_skin_i / f
# Note, no need to check skin MPE, as the time base isn't below 0.25s

In [ ]:
# Note, reduced pulse analysis is not required for skin exposure
mpes_skin = np.array([mpe_s_skin_re, mpe_avg_skin_re])
# minimum mpe per aperture position
mpe_skin_d = np.min(mpes_skin, axis=0)

In [ ]:
## skin hazard distance

In [ ]:
# at each aperture position, compare the emitted radiant exposure with the mpe
# array of accessible emission values
# (ae is vectorised, so can call it with array)
q_skin = ae(q_pulse, w_skin, r_mpe)
a_skin = (w_skin/2)**2 * np.pi
# array of radiant exposure values
re_skin = q_skin / a_skin

shd = r_mpe[np.where(re_skin <= mpe_skin_d)[0][0]]
shd